In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark.sql.functions import col, asc,desc
import time

In [2]:
import os

In [3]:
os.getenv('SPARK_JAVA_OPTS')

In [4]:
filename = "../data/gene_info.tsv"

In [5]:
start = time.time()

In [6]:
showConsoleProgress = False
spark = SparkSession.builder.config("spark.ui.showConsoleProgress", showConsoleProgress).appName("SparkDataframe").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [7]:
column_names = ["GeneID", "Symbol", "Synonyms", "description", "type_of_gene", "#tax_id"]

genes = spark.read.option("header","true").option("sep", "\t").csv(filename)
genes = genes.select(column_names)

In [8]:
genes = genes.filter("type_of_gene == 'protein-coding'")

In [9]:
groups = genes.groupBy(['#tax_id']).count()

In [10]:
groups = groups.toDF("tax_id", "count")

In [11]:
groups = groups.sort(col("count").desc())

Convert Spark to Pandas dataframe (this triggers the computation)

In [12]:
groups = groups.toPandas()

In [13]:
groups.head()

,tax_id,count
0,4565,104033
1,3708,96995
2,90675,82686
3,94328,68154
4,3635,67632


In [14]:
groups.query("tax_id == '9606'")

,tax_id,count
416,9606,20596


### Top 5 organism

In [15]:
groups.head()

,tax_id,count
0,4565,104033
1,3708,96995
2,90675,82686
3,94328,68154
4,3635,67632


In [16]:
spark.stop()

In [17]:
end = time.time()

In [18]:
print("Spark:", end - start, "sec.")

Spark: 52.62713074684143 sec.
